In [327]:
import os
from pandas import Series
import pandasql
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt
from pandas import DataFrame
from scipy.stats import boxcox
import pandas as pd  # data frame operations
import numpy as np  # arrays and math functions
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.stats import uniform  # for training-and-test split
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
import matplotlib.pyplot as plt  # 2D plotting
import seaborn as sns  # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING
sns.set(color_codes=True)
from pandas import read_csv
import matplotlib as mpl
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

In [328]:
# This funtion takes the file and checks & processes
def LoadFile(dataFileName):
    isExisted = os.path.isfile(dataFileName)
    if isExisted == True: # Read in the data when the file does exist
        df = pd.read_csv(dataFileName, encoding = 'latin-1')
    else: # Print our message when the file provide does not exist
        print("File name:", dataFileName, "does not exist in", os.getcwd(), "folder")
    return pd.DataFrame(df)

In [329]:
# Load files by using LoadFile function defines above
states_by_county_death = LoadFile("data/COVID-19_Death_Counts_in_the_United_States_by_County.csv")
state = LoadFile("data/State.csv")
us_counties_cases_deathss = LoadFile("data/us_counties.csv")
us_counties_cases_deaths = LoadFile("data/us_counties_cases_deaths.csv")
detail_2018_county_health_rankings = LoadFile("data/detail_2018_county_health_rankings.csv")
overall_2018_county_health_ranking = LoadFile("data/overall_2018_county_health_ranking.csv")

In [330]:
states_by_county_death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Date as of                 461 non-null    object
 1   First week                 461 non-null    object
 2   Last week                  461 non-null    object
 3   State                      461 non-null    object
 4   County name                461 non-null    object
 5   FIPS County Code           461 non-null    int64 
 6   Deaths involving COVID-19  461 non-null    int64 
 7   Deaths from All Causes     461 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 28.9+ KB


In [331]:
for column in states_by_county_death.columns:
    states_by_county_death.rename(columns={column: column.replace(' ', '_')}, inplace=True)
states_by_county_death.rename(columns={'County_name': 'County_Name'}, inplace=True)

In [332]:
states_by_county_death['County_Name'] = states_by_county_death['County_Name'].str.replace(' County', '')

In [333]:
states_by_county_death.head()

,Date_as_of,First_week,Last_week,State,County_Name,FIPS_County_Code,Deaths_involving_COVID-19,Deaths_from_All_Causes
0,5/20/2020,2/1/2020,5/16/2020,AL,Etowah,1055,11,432
1,5/20/2020,2/1/2020,5/16/2020,AL,Houston,1069,24,542
2,5/20/2020,2/1/2020,5/16/2020,AL,Jefferson,1073,79,2675
3,5/20/2020,2/1/2020,5/16/2020,AL,Lee,1081,43,362
4,5/20/2020,2/1/2020,5/16/2020,AL,Madison,1089,10,1149


In [334]:
states_by_county_death['FIPS_County_Code'] = states_by_county_death['FIPS_County_Code'].astype(str)
states_by_county_death['FIPS_County_Code'] = states_by_county_death['FIPS_County_Code'].str.pad(width=5, side='left', fillchar='0')

In [335]:
states_by_county_death_agg = pandasql.sqldf("SELECT sc.State State, sc.County_Name, sc.FIPS_County_Code [FIPS_Code], sc.[Deaths_involving_COVID-19] [COVID_Death], sc.Deaths_from_All_Causes [All_Cause_Death] FROM states_by_county_death sc")

In [336]:
states_by_county_death.head()

,Date_as_of,First_week,Last_week,State,County_Name,FIPS_County_Code,Deaths_involving_COVID-19,Deaths_from_All_Causes
0,5/20/2020,2/1/2020,5/16/2020,AL,Etowah,01055,11,432
1,5/20/2020,2/1/2020,5/16/2020,AL,Houston,01069,24,542
2,5/20/2020,2/1/2020,5/16/2020,AL,Jefferson,01073,79,2675
3,5/20/2020,2/1/2020,5/16/2020,AL,Lee,01081,43,362
4,5/20/2020,2/1/2020,5/16/2020,AL,Madison,01089,10,1149


In [337]:
states_by_county_death_agg = pandasql.sqldf("SELECT sc.*, s.StateName State_Name FROM states_by_county_death_agg sc LEFT JOIN state s ON rtrim(sc.State) = rtrim(ltrim(s.State))")

In [338]:
states_by_county_death[states_by_county_death['County_Name'] == 'Nassau']

,Date_as_of,First_week,Last_week,State,County_Name,FIPS_County_Code,Deaths_involving_COVID-19,Deaths_from_All_Causes
302,5/20/2020,2/1/2020,5/16/2020,NY,Nassau,36059,2430,6804


In [339]:
states_by_county_death_agg.head()

,State,County_Name,FIPS_Code,COVID_Death,All_Cause_Death,State_Name
0,AL,Etowah,01055,11,432,Alabama
1,AL,Houston,01069,24,542,Alabama
2,AL,Jefferson,01073,79,2675,Alabama
3,AL,Lee,01081,43,362,Alabama
4,AL,Madison,01089,10,1149,Alabama


In [340]:
us_counties_cases_deaths.rename(columns={'fips': 'FIPS_Code'}, inplace=True)
us_counties_cases_deaths.rename(columns={'state': 'State_Name'}, inplace=True)
us_counties_cases_deaths.rename(columns={'county': 'County_Name'}, inplace=True)
us_counties_cases_deaths.rename(columns={'date': 'Date_Name'}, inplace=True)

In [341]:
us_counties_cases_deaths.head()

,Date_Name,County_Name,State_Name,FIPS_Code,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [342]:
us_counties_cases_deaths['FIPS_Code'] = us_counties_cases_deaths['FIPS_Code'].astype(str)
us_counties_cases_deaths['FIPS_Code'] = us_counties_cases_deaths['FIPS_Code'].str.replace('.0', '')
us_counties_cases_deaths['FIPS_Code'] =  us_counties_cases_deaths['FIPS_Code'].str.pad(width=5, side='left', fillchar='0')

In [343]:
us_counties_cases_deaths.head()

,Date_Name,County_Name,State_Name,FIPS_Code,cases,deaths
0,2020-01-21,Snohomish,Washington,00561,1,0
1,2020-01-22,Snohomish,Washington,00561,1,0
2,2020-01-23,Snohomish,Washington,00561,1,0
3,2020-01-24,Cook,Illinois,00131,1,0
4,2020-01-24,Snohomish,Washington,00561,1,0


In [344]:
overall_2018_county_health_ranking.rename(columns={'ï»¿FIPS': 'FIPS_Code'}, inplace=True)
overall_2018_county_health_ranking.rename(columns={'ï»¿FIPS': 'FIPS_Code'}, inplace=True)

In [345]:
us_counties_cases_deaths_agg = pandasql.sqldf("SELECT s.State, c.State_Name, c.County_Name, c.FIPS_Code, SUM(c.cases) COVID_Confirm, SUM(c.deaths) COVID_Death FROM us_counties_cases_deaths c LEFT JOIN state s ON c.State_Name = s.StateName GROUP BY c.State_Name, c.County_Name, c.FIPS_Code")

In [346]:
us_counties_cases_deaths_agg.head()

,State,State_Name,County_Name,FIPS_Code,COVID_Confirm,COVID_Death
0,AL,Alabama,Autauga,00001,2717,123
1,AL,Alabama,Baldwin,00003,7949,197
2,AL,Alabama,Barbour,00005,1981,24
3,AL,Alabama,Bibb,00007,1689,15
4,AL,Alabama,Blount,00009,1590,6


In [347]:
us_counties_cases_deaths.tail()

,Date_Name,County_Name,State_Name,FIPS_Code,cases,deaths
167834,2020-05-22,Sublette,Wyoming,00535,3,0
167835,2020-05-22,Sweetwater,Wyoming,00537,25,0
167836,2020-05-22,Teton,Wyoming,00539,100,1
167837,2020-05-22,Uinta,Wyoming,00541,13,0
167838,2020-05-22,Washakie,Wyoming,00543,19,1


In [348]:
us_counties_cases_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167839 entries, 0 to 167838
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Date_Name    167839 non-null  object
 1   County_Name  167839 non-null  object
 2   State_Name   167839 non-null  object
 3   FIPS_Code    167839 non-null  object
 4   cases        167839 non-null  int64 
 5   deaths       167839 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 7.7+ MB


In [349]:
COVID_19_522 = pandasql.sqldf("SELECT s.State, c.State_Name, c.County_Name, c.FIPS_Code, c.cases COVID_Confirm, c.deaths COVID_Death FROM us_counties_cases_deaths c JOIN state s ON c.State_Name = s.StateName WHERE Date_Name = '2020-05-22'")

In [350]:
COVID_19_522.head()

,State,State_Name,County_Name,FIPS_Code,COVID_Confirm,COVID_Death
0,AL,Alabama,Autauga,00001,149,3
1,AL,Alabama,Baldwin,00003,271,9
2,AL,Alabama,Barbour,00005,104,1
3,AL,Alabama,Bibb,00007,55,1
4,AL,Alabama,Blount,00009,49,1


In [351]:
#pandasql.sqldf("SELECT DISTINCT State, State_Name FROM COVID_19")

In [352]:
us_counties_cases_deaths.rename(columns={'date': 'Date_Name'}, inplace=True)

In [353]:
us_counties_cases_deaths.head()

,Date_Name,County_Name,State_Name,FIPS_Code,cases,deaths
0,2020-01-21,Snohomish,Washington,00561,1,0
1,2020-01-22,Snohomish,Washington,00561,1,0
2,2020-01-23,Snohomish,Washington,00561,1,0
3,2020-01-24,Cook,Illinois,00131,1,0
4,2020-01-24,Snohomish,Washington,00561,1,0


In [354]:
COVID_19_BY_STATE = pandasql.sqldf("SELECT State, SUM(COVID_Confirm) COVID_Confirm, SUM(COVID_Death) COVID_Death FROM COVID_19_522 GROUP BY State")

In [355]:
COVID_19_BY_STATE.head()

,State,COVID_Confirm,COVID_Death
0,AK,404,8
1,AL,13670,541
2,AR,5612,113
3,AZ,15608,775
4,CA,90801,3690


In [356]:
data = dict (
    type = 'choropleth',
    locations = COVID_19_BY_STATE['State'],
    locationmode='USA-states',
    colorscale = ['#3f39bd', '#4ae5f0','#4bd673','#c2c43d'],
    colorbar_title = 'Total',
    #text = COVID_19_BY_STATE['FIPS_Code'],
     showlegend = True,
    z=COVID_19_BY_STATE['COVID_Confirm'])
    
layout = dict(
        title = 'Confirmed Cases',
        geo = dict(scope='usa'))

map = go.Figure(data=[data], layout = layout)
py.iplot(map)

In [357]:
data = dict (
    type = 'choropleth',
    locations = COVID_19_BY_STATE['State'],
    locationmode='USA-states',
    colorscale = 'REDS',
    colorbar_title = 'Total',
     showlegend = True,
    z=COVID_19_BY_STATE['COVID_Death'])
    
layout = dict(
        title = 'Confirmed COVID Death',
        geo = dict(scope='usa'))

map = go.Figure(data=[data], layout = layout)
py.iplot(map)

In [358]:
data = dict (
    type = 'choropleth',
    locations = states_by_county_death['State'],
    locationmode='USA-states',
    colorscale = ['#3f39bd', '#4ae5f0','#4bd673','#c2c43d'],
    colorbar_title = 'Total',
     #text = COVID_19['FIPS_Code'],
     showlegend = True,
     z=states_by_county_death['Deaths_from_All_Causes'])
    
layout = dict(
        title = 'All Cause of Death',
        geo = dict(scope='usa'))

map = go.Figure(data=[data], layout = layout)
py.iplot(map)

In [359]:
overall_2018_county_health_ranking['FIPS_Code'] = overall_2018_county_health_ranking['FIPS_Code'].astype(str)
overall_2018_county_health_ranking['FIPS_Code'] = overall_2018_county_health_ranking['FIPS_Code'].str.pad(width=5, side='left', fillchar='0')

In [360]:
overall_2018_county_health_ranking.head()

,FIPS_Code,State,County,Length of Life Rank,Quality of Life Rank,Health Behaviors Rank,Clinical Care Rank,Social & Economic Factors Rank,Physical Environment Rank,Health Outcomes Rank,Health Factors Rank
0,01001,Alabama,Autauga,18,5,12,15,3,51,11,8
1,01003,Alabama,Baldwin,4,4,3,7,8,14,3,3
2,01005,Alabama,Barbour,14,49,57,20,58,16,34,56
3,01007,Alabama,Bibb,53,25,39,42,42,29,41,37
4,01009,Alabama,Blount,17,12,11,41,13,54,14,19


In [361]:
detail_2018_county_health_rankings.rename(columns={'FIPS': 'FIPS_Code'}, inplace=True)
detail_2018_county_health_rankings.rename(columns={'State': 'StateName'}, inplace=True)

In [362]:
detail_2018_county_health_rankings['FIPS_Code'] = detail_2018_county_health_rankings['FIPS_Code'].astype(str)
detail_2018_county_health_rankings['FIPS_Code'] = detail_2018_county_health_rankings['FIPS_Code'].str.pad(width=5, side='left', fillchar='0')

In [363]:
detail_2018_county_health_rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 56 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   FIPS_Code                           3142 non-null   object 
 1   StateName                           3142 non-null   object 
 2   County                              3142 non-null   object 
 3   Number_Adult_Deaths                 3077 non-null   float64
 4   Age_Adjusted_Mortality              3077 non-null   float64
 5   Age_Adjusted_Mortality_Black        1334 non-null   float64
 6   Age_Adjusted_Mortality_Hispanic     797 non-null    float64
 7   Age_Adjusted_Mortality_White        1556 non-null   float64
 8   Number_Child_Deaths                 1931 non-null   float64
 9   Child_Mortality_Rate                1931 non-null   float64
 10  Child_Mortality_Rate_Black          498 non-null    float64
 11  Child_Mortality_Rate_Hispanic       359 non

In [364]:
detail_2018_county_health_rankings.head()

,FIPS_Code,StateName,County,Number_Adult_Deaths,Age_Adjusted_Mortality,Age_Adjusted_Mortality_Black,Age_Adjusted_Mortality_Hispanic,Age_Adjusted_Mortality_White,Number_Child_Deaths,Child_Mortality_Rate,...,Percent_African_American,Percent_American_Alaskan_Indian,Percent_Asian,Percent_Hawaiian_PacificIslander,Percent_Hispanic,Percent_White,Percent_ Not_Proficient_English,Percent_Female,Percent_Rural,Unnamed: 55
0,01001,Alabama,Autauga,827.0,450.0,532.0,NaN,445.0,31.0,56.2,...,19.0,0.5,1.1,0.1,2.7,75.0,0.8,51.3,42.0,NaN
1,01003,Alabama,Baldwin,2753.0,351.0,518.0,111.0,349.0,85.0,47.7,...,9.0,0.8,1.1,0.1,4.4,83.2,0.8,51.5,42.3,NaN
2,01005,Alabama,Barbour,418.0,434.0,477.0,NaN,395.0,15.0,66.6,...,47.5,0.6,0.5,0.2,4.2,46.5,1.5,46.9,67.8,NaN
3,01007,Alabama,Bibb,408.0,529.0,630.0,NaN,527.0,22.0,116.3,...,21.6,0.4,0.2,0.1,2.5,74.4,0.4,46.4,68.4,NaN
4,01009,Alabama,Blount,996.0,480.0,NaN,NaN,NaN,30.0,55.4,...,1.4,0.6,0.3,0.1,9.1,87.5,1.9,50.6,90.0,NaN


In [365]:
for column in overall_2018_county_health_ranking.columns:
    overall_2018_county_health_ranking.rename(columns={column: column.replace(' ', '_').replace('&_', '')}, inplace=True)
    
overall_2018_county_health_ranking.rename(columns={'State': 'State_Name'}, inplace=True)
overall_2018_county_health_ranking.rename(columns={'County': 'County_Name'}, inplace=True)

In [366]:
overall_2018_county_health_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   FIPS_Code                     3142 non-null   object
 1   State_Name                    3142 non-null   object
 2   County_Name                   3142 non-null   object
 3   Length_of_Life_Rank           3142 non-null   object
 4   Quality_of_Life_Rank          3142 non-null   object
 5   Health_Behaviors_Rank         3142 non-null   object
 6   Clinical_Care_Rank            3142 non-null   object
 7   Social_Economic_Factors_Rank  3142 non-null   object
 8   Physical_Environment_Rank     3142 non-null   object
 9   Health_Outcomes_Rank          3142 non-null   object
 10  Health_Factors_Rank           3142 non-null   object
dtypes: object(11)
memory usage: 270.1+ KB


In [367]:
overall_2018_county_health_ranking.head()

,FIPS_Code,State_Name,County_Name,Length_of_Life_Rank,Quality_of_Life_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,Health_Outcomes_Rank,Health_Factors_Rank
0,01001,Alabama,Autauga,18,5,12,15,3,51,11,8
1,01003,Alabama,Baldwin,4,4,3,7,8,14,3,3
2,01005,Alabama,Barbour,14,49,57,20,58,16,34,56
3,01007,Alabama,Bibb,53,25,39,42,42,29,41,37
4,01009,Alabama,Blount,17,12,11,41,13,54,14,19


In [368]:
COVID_19_522_OVERALL_HEALTH = pandasql.sqldf("SELECT c.State, c.State_Name, c.County_Name, c.FIPS_Code, c.COVID_Confirm, c.COVID_Death, o.Length_of_Life_Rank, o.Quality_of_Life_Rank, o.Health_Behaviors_Rank, o.Clinical_Care_Rank, o.Social_Economic_Factors_Rank, o.Physical_Environment_Rank,	o.Health_Outcomes_Rank,	o.Health_Factors_Rank FROM COVID_19_522 c LEFT JOIN overall_2018_county_health_ranking o ON c.State_Name = o.State_Name")

In [369]:
COVID_19_522_OVERALL_HEALTH.fillna(0)

,State,State_Name,County_Name,FIPS_Code,COVID_Confirm,COVID_Death,Length_of_Life_Rank,Quality_of_Life_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,Health_Outcomes_Rank,Health_Factors_Rank
0,AL,Alabama,Autauga,00001,149,3,1,1,1,1,1,47,1,1
1,AL,Alabama,Autauga,00001,149,3,10,18,9,8,7,32,13,6
2,AL,Alabama,Autauga,00001,149,3,11,34,35,5,18,19,16,13
3,AL,Alabama,Autauga,00001,149,3,12,8,13,13,11,39,9,10
4,AL,Alabama,Autauga,00001,149,3,13,13,10,24,12,57,12,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283833,WY,Wyoming,Washakie,00543,19,1,5,14,7,22,16,5,8,15
283834,WY,Wyoming,Washakie,00543,19,1,6,18,20,11,20,19,15,21
283835,WY,Wyoming,Washakie,00543,19,1,7,16,14,15,9,20,14,11
283836,WY,Wyoming,Washakie,00543,19,1,8,9,2,13,4,12,7,4


In [370]:
for column in COVID_19_522_OVERALL_HEALTH:
    COVID_19_522_OVERALL_HEALTH[column] = COVID_19_522_OVERALL_HEALTH[column].astype(str).replace('NR', '0')

In [371]:
COVID_19_522_OVERALL_HEALTH['COVID_Confirm'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['COVID_Confirm'])
COVID_19_522_OVERALL_HEALTH['COVID_Death'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['COVID_Death'])
COVID_19_522_OVERALL_HEALTH['Length_of_Life_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Length_of_Life_Rank'])
COVID_19_522_OVERALL_HEALTH['Quality_of_Life_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Quality_of_Life_Rank'])
COVID_19_522_OVERALL_HEALTH['Health_Behaviors_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Health_Behaviors_Rank'])
COVID_19_522_OVERALL_HEALTH['Clinical_Care_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Clinical_Care_Rank'])
COVID_19_522_OVERALL_HEALTH['Social_Economic_Factors_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Social_Economic_Factors_Rank'])
COVID_19_522_OVERALL_HEALTH['Physical_Environment_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Physical_Environment_Rank'])
COVID_19_522_OVERALL_HEALTH['Health_Outcomes_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Health_Outcomes_Rank'])
COVID_19_522_OVERALL_HEALTH['Health_Factors_Rank'] = pd.to_numeric(COVID_19_522_OVERALL_HEALTH['Health_Factors_Rank'])

In [372]:
COVID_19_522_OVERALL_HEALTH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283838 entries, 0 to 283837
Data columns (total 14 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   State                         283838 non-null  object
 1   State_Name                    283838 non-null  object
 2   County_Name                   283838 non-null  object
 3   FIPS_Code                     283838 non-null  object
 4   COVID_Confirm                 283838 non-null  int64 
 5   COVID_Death                   283838 non-null  int64 
 6   Length_of_Life_Rank           283838 non-null  int64 
 7   Quality_of_Life_Rank          283838 non-null  int64 
 8   Health_Behaviors_Rank         283838 non-null  int64 
 9   Clinical_Care_Rank            283838 non-null  int64 
 10  Social_Economic_Factors_Rank  283838 non-null  int64 
 11  Physical_Environment_Rank     283838 non-null  int64 
 12  Health_Outcomes_Rank          283838 non-null  int64 
 13 

In [326]:
sns.pairplot(COVID_19_522_OVERALL_HEALTH, x_vars=['Length_of_Life_Rank','Quality_of_Life_Rank', 'Health_Behaviors_Rank', 'Clinical_Care_Rank', 'Social_Economic_Factors_Rank', 'Physical_Environment_Rank', 'Health_Outcomes_Rank', 'Health_Factors_Rank'],  y_vars='COVID_Confirm', height=5, aspect=0.7, kind='reg')

Error in callback <function flush_figures at 0x0000029511AA3C18> (for post_execute):


KeyboardInterrupt: 

In [ ]:
sns.pairplot(COVID_19_522_OVERALL_HEALTH, x_vars=['Length_of_Life_Rank','Quality_of_Life_Rank', 'Health_Behaviors_Rank', 'Clinical_Care_Rank', 'Social_Economic_Factors_Rank', 'Physical_Environment_Rank', 'Health_Outcomes_Rank', 'Health_Factors_Rank'],  y_vars='COVID_Death', height=5, aspect=0.7, kind='reg')